In [1]:
import numpy as np
import matplotlib.pyplot as plt
from solvers import steepest_descent, newton, BFGS, DFP
from objectives import get_rosenbrock, get_lgt_obj, get_zakharov

Common plotting method for all 3 functions

In [ ]:
def plot_results(dim, results, title, filename):
    plt.figure(figsize=(30, 20))  # Increase the size of the plot
    plt.subplot(1, 2, 1)
    for label, (function_history, gradient_norms, cumulative_times) in results.items():
        plt.plot(function_history[:100], label=f'{label} - Function Value') 
    plt.title(f'{title} - Function Value')
    plt.xlabel('Iterations')
    plt.ylabel('Function Value')
    plt.legend(fontsize=26)

    plt.subplot(1, 2, 2)
    for label, (function_history, gradient_norms, cumulative_times) in results.items():
        plt.plot(gradient_norms[:100], label=f'{label} - Gradient Norm')  
    plt.title(f'{title} - Gradient Norm')
    plt.xlabel('Iterations')
    plt.ylabel('Gradient Norm')
    plt.legend(fontsize=26)

    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

Common method to save comparision in number of iterations for each optimization algo

In [3]:
import pandas as pd

def save_results_to_csv(results, filename):

    data = {
        'Algorithm': [],
        'Dimension_or_Lambda': [],
        'Time': [],
        'Final Function Value': [],
        'Final Gradient Norm': [],
        'Iterations': []
    }

    # Populate the dictionary with the results
    for dim, methods_results in results.items():
        for label, (function_history, gradient_norms,cumulative_times) in methods_results.items():
            data['Algorithm'].append(label)
            data['Dimension_or_Lambda'].append(dim)
            data['Time'].append(cumulative_times[-1])
            data['Final Function Value'].append(function_history[-1])
            data['Final Gradient Norm'].append(gradient_norms[-1])
            data['Iterations'].append(len(function_history))
        
    df_results = pd.DataFrame(data)
    df_results.to_csv(filename, index=False)


For Rosebrock

In [21]:
dimensions = [2, 5, 10, 50]
results = {}

for dim in dimensions:
    f, gradf, hessf, x0 = get_rosenbrock(dim)
    results[dim] = {}

    methods = {
        'Steepest Descent': steepest_descent,
        'Newton': newton,
        'BFGS': BFGS,
        'DFP': DFP
    }

    for label, method in methods.items():
        if label == 'Newton':
            xsol, function_history, cumulative_times, gradient_norms = method(x0, f, gradf, hessf)
        else:
            xsol, function_history, cumulative_times, gradient_norms = method(x0, f, gradf)
        results[dim][label] = (function_history, gradient_norms, cumulative_times)
        print(f"{label} done for dim={dim}")

    plot_results(dim, results[dim], f'Rosenbrock Function Optimization for Dimension {dim}', f'rosenbrock_{dim}.png')
save_results_to_csv(results, 'rosenbrock_results.csv')

Steepest Descent done for dim=2
Newton done for dim=2
BFGS done for dim=2
DFP done for dim=2
Steepest Descent done for dim=5
Newton done for dim=5
BFGS done for dim=5
DFP done for dim=5
Steepest Descent done for dim=10
Newton done for dim=10
BFGS done for dim=10
DFP done for dim=10
Steepest Descent done for dim=50
Newton done for dim=50
BFGS done for dim=50
DFP done for dim=50


For Zakharov

In [5]:
results = {}
# Iterate over each dimension

f, gradf, hessf, x0 = get_zakharov(2)
results[0] = {}

methods = {
    'Steepest Descent': steepest_descent,
    'Newton': newton,
    'BFGS': BFGS,
    'DFP': DFP
}

for label, method in methods.items():
    if label == 'Newton':
        xsol, function_history, cumulative_times, gradient_norms = method(x0, f, gradf, hessf)
    else:
        xsol, function_history, cumulative_times, gradient_norms = method(x0, f, gradf)
    results[0][label] = (function_history, gradient_norms, cumulative_times)
    print(f"{label} done for dim={2}")

plot_results(2, results[0], f'Zakharov Function Optimization for Dimension {2}', f'zakharov_2.png')
save_results_to_csv(results, 'zakharov_results.csv')


Steepest Descent done for dim=2
Newton done for dim=2
BFGS done for dim=2
DFP done for dim=2


For Logistic Regression on MNIST

In [22]:
import time
lambdas = [0.001, 0.01, 0.1]
results = {}

for lam in lambdas:
    f, gradf, hessf, x0 = get_lgt_obj(lam)
    results[lam] = {}

    methods = {
        'Steepest Descent': lambda x0, f, gradf: steepest_descent(x0, f, gradf, c0=0.001, c1=0.9, t0=0.1e-3, grad_tol=1e-4),
        'Newton': lambda x0, f, gradf, hessf: newton(x0, f, gradf, hessf, c0=0.001, c1=0.9, t0=1e-3, grad_tol=1e-4),
        'BFGS': lambda x0, f, gradf: BFGS(x0, f, gradf, c0=0.001, c1=0.8, t0=1e-3, grad_tol=1e-3),
        'DFP': lambda x0, f, gradf: DFP(x0, f, gradf, c0=0.001, c1=0.5, t0=1e-3, grad_tol=1e-3)
    }

    for label, method in methods.items():
        print(f"Running {label} for λ={lam}")
        start_time = time.time()
        if label == 'Newton':
            xsol, function_history, cumulative_times, gradient_norms = method(x0, f, gradf, hessf)
        else:
            xsol, function_history, cumulative_times, gradient_norms = method(x0, f, gradf)
        results[lam][label] = (function_history, gradient_norms, cumulative_times)
        end_time = time.time()
        print(f"{label} done for λ={lam} in {end_time - start_time} seconds in cumulative time {cumulative_times[-1]}")

    plot_results(lam, results[lam], f'LGT Function Optimization with λ={lam}', f'lgt_{lam}.png')
    
save_results_to_csv(results, 'lgt_results.csv')


Running Steepest Descent for λ=0.001
Steepest Descent done for λ=0.001 in 23.729028701782227 seconds in cumulative time 226
Running Newton for λ=0.001
Newton done for λ=0.001 in 8.95393681526184 seconds in cumulative time 183
Running BFGS for λ=0.001
BFGS done for λ=0.001 in 14.383190155029297 seconds in cumulative time 409
Running DFP for λ=0.001
DFP done for λ=0.001 in 14.480225086212158 seconds in cumulative time 720
Running Steepest Descent for λ=0.01
Steepest Descent done for λ=0.01 in 18.608142137527466 seconds in cumulative time 237
Running Newton for λ=0.01
Newton done for λ=0.01 in 11.575324058532715 seconds in cumulative time 248
Running BFGS for λ=0.01
BFGS done for λ=0.01 in 10.740325927734375 seconds in cumulative time 304
Running DFP for λ=0.01
DFP done for λ=0.01 in 5.426794052124023 seconds in cumulative time 309
Running Steepest Descent for λ=0.1
Steepest Descent done for λ=0.1 in 4.0390729904174805 seconds in cumulative time 152
Running Newton for λ=0.1
Newton done fo

Tuning c0 and c1 - simply for debugging


In [7]:
import time

# Define search grid
c0_values = [1e-4, 1e-3, 1e-2]  # Typical range

# Fixed parameters
c1_fixed = 0.9  # Fixed c1 value
lambda_fixed = 0.001  # Regularization parameter
t0_fixed = 1e-3  # Small step size
grad_tol_fixed = 1e-4  # Small enough tolerance

# Load LGT problem
f, gradf, hessf, x0 = get_lgt_obj(lambda_fixed)

# Storage for results
results = {}

for c0 in c0_values:
    results[c0] = {}

    # Test Newton method
    method = lambda x, f, gradf, hessf: newton(x, f, gradf, hessf, c0=c0, c1=c1_fixed, t0=t0_fixed, grad_tol=grad_tol_fixed)

    start_time = time.time()
    xsol, function_history, cumulative_times, gradient_norms = method(x0, f, gradf, hessf)
    end_time = time.time()

    results[c0]["Newton"] = {
        "iterations": len(function_history),
        "final_f": function_history[-1],
        "final_grad_norm": gradient_norms[-1]
    }

    print(f"Newton done for c0={c0} in {end_time - start_time} seconds")
    print(f"Iterations: {len(function_history)}, Final Function Value: {function_history[-1]}, Final Gradient Norm: {gradient_norms[-1]}")

# Convert to a heatmap-friendly format
data = []
for c0, methods in results.items():
    for method, values in methods.items():
        data.append([method, c0, values["iterations"], values["final_f"], values["final_grad_norm"]])

df = pd.DataFrame(data, columns=["Method", "c0", "Iterations", "Final Function Value", "Final Gradient Norm"])

print(df)


Newton done for c0=0.0001 in 8.677034854888916 seconds
Iterations: 184, Final Function Value: 0.031271444413236635, Final Gradient Norm: 9.652822419282265e-05
Newton done for c0=0.001 in 9.342303037643433 seconds
Iterations: 184, Final Function Value: 0.031271444413236635, Final Gradient Norm: 9.652822419282265e-05
Newton done for c0=0.01 in 9.408971071243286 seconds
Iterations: 184, Final Function Value: 0.031271444413236635, Final Gradient Norm: 9.652822419282265e-05
   Method      c0  Iterations  Final Function Value  Final Gradient Norm
0  Newton  0.0001         184              0.031271             0.000097
1  Newton  0.0010         184              0.031271             0.000097
2  Newton  0.0100         184              0.031271             0.000097
